# Analyse der Entitaeten (Named Entity Recognition)

In [1]:
#Import Bibliothek
import pandas as pd
import spacy
from collections import Counter
import matplotlib.pyplot as plt

C:\Users\MarkE\AppData\Roaming\Python\Python310\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
#Abruf Daten 
file_path = r'2_Inhaltsanalyse_nach_Sentiment.csv'
df = pd.read_csv(file_path)
print(df.head())

  Quelle       Datum                                               Link  \
0    FAZ  11/30/2023  https://www.faz.net/aktuell/wirtschaft/unterne...   
1    FAZ  11/30/2023  https://www.faz.net/aktuell/wirtschaft/kuenstl...   
2    FAZ  11/29/2023  https://www.faz.net/aktuell/feuilleton/medien/...   
3    FAZ  11/28/2023  https://www.faz.net/pro/d-economy/kuenstliche-...   
4    FAZ  11/28/2023  https://www.faz.net/pro/d-economy/prompt-der-w...   

                                               Titel  \
0  KI: Warum wir nicht mit Roboter-Autos vollauto...   
1  Microsoft: Sind Jahrzehnte von einer künstlich...   
2  Künstliche Intelligenz: Europa muss von neuen ...   
3  Was die Superintelligenz-KI anrichten könnte, ...   
4  Künstliche Intelligenz: Wie man sich seine Pro...   

                                                Text  Anzahl Woerter Text  \
0  Roboterautos faszinieren viele – die Augen der...                 1963   
1  Der Krieg von Mensch gegen Maschine verschiebt...      

In [3]:
#Analyse der Entitaeten (NER) basierend auf den bereinigten Daten

#Laden des englische Sprachmodells für spaCy
nlp = spacy.load("en_core_web_sm")

def extract_ner(text):
    doc = nlp(text)
    entities_with_types = [f"{ent.label_}: {ent.text}" for ent in doc.ents]
    return entities_with_types

#Anwendung der Funktion auf den DataFrame
df['NER_entitaeten'] = df['Text_bereinigt'].apply(extract_ner)

In [4]:
print('Originaltext: ',df.loc[35,'Text'])
print('Bereinigter Text: ',df.loc[35,'Text_bereinigt'])
print('Entitäten: ',df.loc[35,'NER_entitaeten'])

Originaltext:  Bill Gates erwartet eine Revolution der digitalen Welt: Menschen werden mit Computern in natürlicher Sprache sprechen – und die Computer werden es dank KI auch verstehen. „Ich liebe Software heute genauso sehr wie damals, als Paul Allen und ich Microsoft gründeten. Aber – obwohl sie sich in den Jahrzehnten seitdem stark verbessert hat – ist Software in vielerlei Hinsicht immer noch ziemlich dumm“, beginnt Gates seinen Blogbeitrag, in dem er den nächsten großen Schritt des Computings beschreibt: Die Bewegung von Apps zu Ki-Agenten als persönliche digitale Assistenten. Wir dokumentieren hier wesentliche Passagen seines Textes. „Um eine Aufgabe auf einem Computer zu erledigen, müssen Sie Ihrem Gerät sagen, welche App Sie verwenden sollen. Sie können Microsoft Word und Google Docs verwenden, um einen Geschäftsvorschlag zu entwerfen, aber sie können Ihnen nicht helfen, eine E-Mail zu senden, ein Selfie zu teilen, Daten zu analysieren, eine Party zu planen oder Kinokarten zu k

In [5]:
#Manuelle Korrekturen der Entitaeten

#Sicherstellen, dass alle Werte in 'NER_entitaeten' als String behandelt werden
df['NER_entitaeten'] = df['NER_entitaeten'].astype(str)

#Entfernen aller Leerzeichen vor und nach Kommas sowie vor und nach Doppelpunkten
df['NER_entitaeten'] = df['NER_entitaeten'].str.replace(r'\s*,\s*', ',', regex=True)
df['NER_entitaeten'] = df['NER_entitaeten'].str.replace(r'\s*:\s*', ':', regex=True)


#Liste der zu ersetzenden Begriffe und ihre neuen Werte
replacements = {
    'ORG:AI': 'MISC:AI',
    'ORG:Chatgpt': 'PRODUCT:Chatgpt',
    'ORG:GPT': 'PRODUCT:GPT',
    'PERSON:Openaai': 'ORG:Openai',
    'LAW:GPT 4': 'PRODUCT:GPT 4',
    'PERSON:Ki': 'MISC:AI',
    'ORG:Bard': 'PRODUCT:Bard',  
    'ORG:Google Bard': 'PRODUCT:Bard',  
    'PRODUCT:Altman': 'PERSON:Sam Altman', 
    'ORG:KI': 'MISC:AI',
    'PERSON:Twitter': 'ORG:Twitter',
    'GPE:Alphabet': 'ORG:Alphabet',
    'ORG:YouTube': 'PRODUCT:YouTube', 
    'PERSON:AI': 'MISC:AI',
    'ORG:KIS': 'MISC:AI',
    'ORG:Android': 'PRODUCT:Android',
    'LOC:Corona': 'EVENT:Corona', 
    'GPE:AI': 'MISC:AI',
    'PERSON:Alph Alpha': 'ORG:Aleph Alpha',
    'PERSON:Open Ai': 'ORG:Openai',
    'GPE:Alpha': 'ORG:Aleph Alpha',
    'LAW:GPT 3': 'PRODUCT:GPT 3',
    'PERSON:Bard': 'PRODUCT:Bard',
    'EVENT:AI': 'MISC:AI',
    'FAC:Openai': 'ORG:Openai',
    'FAC:Open Ai': 'ORG:Openai',
    'FAC:Alph Alpha': 'ORG:Aleph Alpha',
    'FAC:Reinforcement Learning': 'MISC:Reinforcement Learning',
    'FAC:Lake Constance': 'LOC:Lake Constance',
    'FAC:Countries Great Britain': 'GPE:Great Britain',
    'GPE:Wiaipedia': 'PRODUCT:Wikipedia',
    'GPE:Hesse': 'NORP:Hesse',
    'GPE:Abitur': 'EVENT:Abitur',
    'GPE:Nvidia': 'ORG:Nvidia',
    'GPE:Netflix': 'ORG:Netflix',
    'GPE:Github': 'ORG:Github',
    'GPE:Tikkok': 'ORG:Tiktok',
    'GPE:Deepmind': 'ORG:Deepmind',
    'GPE:LinkedIn': 'PRODUCT:LinkedIn',
    'GPE:Obama': 'PERSON:Obama',
    'LOC:Obama': 'PERSON:Obama',
    'GPE:Openai': 'ORG:Openai',
    'GPE:Haaland': 'PERSON:Haaland',
    'GPE:Network Germany': 'GPE:Germany',
    'LANGUAGE:english': 'LANGUAGE:English',
    'LAW:AI Act AI Act': 'LAW:AI Act',
    'LOC:Heil': 'PERSON:Heil',
    'LOC:Manta Manta 2': 'WORKOFART:Manta Manta 2',
    'LOC:Instragram': 'PRODUCT:Instragram',
    'MISC:AIS': 'MISC:AI',
    'MISC:AI AI': 'MISC:AI',
    'NORP:German': 'NORP:german',
    'NORP:Germans': 'NORP:german',
    'NORP:Americans': 'NORP:american',
    'NORP:American': 'NORP:american',
    'NORP:English': 'NORP:nglish',
    'NORP:British': 'NORP:british',
    'NORP:French': 'NORP:french',
    'NORP:Chinese': 'NORP:chinese',
    'NORP:Spanish': 'NORP:spanish',
    'NORP:Altmans': 'PERSON:Sam Altman',
    'NORP:Andrulis': 'PERSON:Jonas Andrulis',
    'NORP:Windows': 'PRODUCT:Windows',
    'NORP:Bundesliga': 'PRODUCT:Bundesliga',
    'ORG:Altman': 'PERSON:Sam Altman',
    'ORG:Bing': 'PRODUCT:Bing',
    'ORG:Alexa': 'PRODUCT:Alexa',
    'ORG:Instragram': 'PRODUCT:Instragram',
    'ORG:Heidelberg': 'GPE:Heidelberg',
    'ORG:White House': 'FAC:White House',
    'ORG:gpt': 'PRODUCT:GPT',
    'ORG:iPhone': 'PRODUCT:iPhone',
    'ORG:Trump': 'PERSON:Donald Trump',
    'ORG:Olaf Scholz': 'PERSON:Olaf Scholz',
    'ORG:Microsoft Google': '',
    'ORG:Chatbot Chatgpt': '',
    'ORG:LLM': 'MISC:LLM',
    'ORG: Gates': 'PERSON: Gates',
    'ORG:Deepl': 'PRODUCT:Deepl',
    'ORG:Openai Microsoft': '',
    'ORG:BARD': 'PRODUCT:Bard',
    'ORG:Sascha Lobo': 'PERSON:Sascha Lobo',
    'ORG:Olaf Scholz': 'PERSON:Ilya Sutskever',
    'ORG:Volkswagen': 'ORG:VW',
    'ORG:Brockman': 'PERSON:Greg Brockman',
    'ORG:Corona': 'EVENT:Corona',
    'PERSON:Musk': 'PERSON:Elon Musk',
    'PERSON:Midjourney': 'ORG:Midjourney',
    'PERSON:Nadella': 'PERSON:Satya Nadella',
    'PERSON:Pichai': 'PERSON:Sundar Pichai',
    'PERSON:Andrulis': 'PERSON:Jonas Andrulis',
    'PERSON:Habeck': 'PERSON:Robert Habeck',
    'PERSON:Putin': 'PERSON:Vladimir Putin',
    'PERSON:Pixel': 'PRODUCT:Pixel',
    'PERSON:Bloomberg': 'ORG:Bloomberg',
    'PERSON:Biden': 'PERSON:Joe Biden',
    'PERSON:Zuckerberg': 'PERSON:Mark Zuckerberg',
    'PERSON:Generative AI': 'MISC:Generative AI',
    'PERSON:KI': 'MISC:AI',
    'PERSON:Ursula von der': 'PERSON:Ursula von der Leyen',
    'PERSON:Baden Wrttemberg': 'LOC:Baden Wrttemberg',
    'PERSON:Tikok': 'ORG:Tiktok',
    'PERSON:Aleph Alpha': 'ORG:Aleph Alpha',
    'PERSON:Alphabet': 'ORG:Alphabet',
    'PERSON:Lidl': 'ORG:Lidl',
    'PERSON:Hessen': 'GPE:Hessen',
    'PERSON:Dsseldorf': 'GPE:Dsseldorf',
    'PERSON:Brandenburg': 'GPE:Brandenburg',
    'PRODUCT:Elon Musk': 'PERSON:Elon Musk',
    'PRODUCT:Chatgpt Co': 'PRODUCT:Chatgpt',
    'PRODUCT:Chatgpt AI': 'PRODUCT:Chatgpt',
    'PRODUCT:Chatgpt Openai': 'PRODUCT:Chatgpt',
    'PRODUCT:Bard Google': 'PRODUCT:Bard',
    'PRODUCT:Alpha': 'ORG:Aleph Alpha',
    'PRODUCT:Aleph Alpha': 'ORG:Aleph Alpha',
    'PRODUCT:Chatgpt Google': '',
    'WORKOFART:Nobel': 'WORKOFART:Nobel Prize',
    'WORKOFART:Open Ai': 'ORG:Openai',
    'WORKOFART:AI': 'MISC:AI',
    'GPE:Gemini': 'PRODUCT:Gemini',
    'PERSON:Gemini': 'PRODUCT:Gemini',
    'ORG:Gemini': 'PRODUCT:Gemini',
    'ORG:Google Gemini': 'PRODUCT:Gemini',
    'PERSON:Axel Springer': 'ORG:Axel Springer',
    'MISC:AI Act': 'LAW:AI Act',
    'MISC:AI VO': 'LAW:AI Act',
    'ORG: WhatsApp':"PRODUCT: WhatsApp"
}

#Funktion, die die Ersetzungen aus der Liste anwendet
def apply_replacements(text):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

#Anwenden der Ersetzungen auf die Spalte 'NER_entitaeten'
df['NER_entitaeten'] = df['NER_entitaeten'].apply(apply_replacements)

#Anzeigen des aktualisierten DataFrames
print(df)

     Quelle       Datum                                               Link  \
0       FAZ  11/30/2023  https://www.faz.net/aktuell/wirtschaft/unterne...   
1       FAZ  11/30/2023  https://www.faz.net/aktuell/wirtschaft/kuenstl...   
2       FAZ  11/29/2023  https://www.faz.net/aktuell/feuilleton/medien/...   
3       FAZ  11/28/2023  https://www.faz.net/pro/d-economy/kuenstliche-...   
4       FAZ  11/28/2023  https://www.faz.net/pro/d-economy/prompt-der-w...   
...     ...         ...                                                ...   
2044   Zeit   12/6/2023  https://www.zeit.de/news/2023-12/06/google-wil...   
2045   Zeit   12/4/2023  https://www.zeit.de/digital/2023-11/ki-gesetz-...   
2046   Zeit   12/3/2023  https://www.zeit.de/politik/ausland/2023-12/is...   
2047   Zeit   12/2/2023  https://www.zeit.de/2023/51/kuenstliche-intell...   
2048   Zeit   12/2/2023  https://www.zeit.de/news/2023-12/02/neuer-poli...   

                                                  Titel  \
0   

In [6]:
#Liste an Entitaeten

#Sicherstellen, dass alle Werte in der Spalte als Strings behandelt werden
df['NER_entitaeten'] = df['NER_entitaeten'].astype(str)

#Woerter sind durch Kommas getrennt
words = df['NER_entitaeten'].str.split(',').explode().str.strip()

#Zähle die Woerter mit Counter
word_counts = Counter(words)

#Abrufen und Sortieren der häufigsten Woerter
most_common_words = word_counts.most_common()

#Umwandlung der Liste der häufigsten Woerter in einen DataFrame
df_most_common_words = pd.DataFrame(most_common_words, columns=['Wort', 'Anzahl'])

#Trennung der Eintraege in der Spalte 'Wort' in zwei Teile, sicherstellen, dass immer zwei Teile zurückgegeben werden
df_most_common_words[['Typ', 'Wort']] = df_most_common_words['Wort'].apply(
    lambda x: pd.Series(x.split(':', 1) if ':' in x else [x, '']))

#Sortieren des DataFrame nach 'Anzahl im Text' in absteigender Reihenfolge
df_most_common_words = df_most_common_words.sort_values(by='Anzahl', ascending=False)

#Zuruecksetzen des Index
df_most_common_words = df_most_common_words.reset_index(drop=True)

#Entfernen von Sonderzeichen Spalten 'Typ' und 'Wert'
df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#Entfernen von Apostrophen in den Spalten 'Wort-Typ' und 'Wort-Wert'
df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace("'", "")
df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace("'", "")

#Filtern der Eintraege, wo sowohl 'Typ' als auch 'WWert' einen Eintrag haben
df_most_common_words = df_most_common_words[(df_most_common_words['Typ'] != '') & (df_most_common_words['Wort'] != '')]

#Neuordnen des DataFrame nach den Spalten 'Typ', 'Wort' und 'Anzahl'
df_most_common_words = df_most_common_words[['Typ', 'Wort', 'Anzahl']]

#Speichern des sortierten DataFrame in eine CSV-Datei
df_most_common_words.to_csv('4_NER_Entitaeten.csv', index=False)

#Oberen Teil des DataFrame anzeigen, um die Änderungen zu überprüfen
print(df_most_common_words.head(20))

         Typ        Wort  Anzahl
0       MISC          AI   10767
1    PRODUCT     Chatgpt    1573
2    ORDINAL       first    1481
3   CARDINAL         one    1287
4        GPE     Germany    1214
5        ORG   Microsoft    1108
6        ORG      Google     993
7       NORP      german     827
8        ORG      Openai     746
9   CARDINAL         two     725
10      DATE       today     626
11    PERSON  Sam Altman     597
12       GPE          US     587
13       ORG          EU     518
14       LOC      Europe     433
15   ORDINAL      second     432
16   PRODUCT         GPT     422
17  CARDINAL       three     419
18      NORP    european     356
19      NORP    american     352


In [7]:
#Liste an Entitaeten vor ChatGPT
df_vor_ChatGPT = df[df['Zeitraum'] == '0. vor ChatGPT']

#Sicherstellen, dass alle Werte in der Spalte als Strings behandelt werden
df_vor_ChatGPT['NER_entitaeten'] = df_vor_ChatGPT['NER_entitaeten'].astype(str)

#Woerter sind durch Kommas getrennt
words = df_vor_ChatGPT['NER_entitaeten'].str.split(',').explode().str.strip()

#Zähle die Woerter mit Counter
word_counts = Counter(words)

#Abrufen und Sortieren der häufigsten Woerter
most_common_words = word_counts.most_common()

#Umwandlung der Liste der häufigsten Woerter in einen DataFrame
df_most_common_words = pd.DataFrame(most_common_words, columns=['Wort', 'Anzahl'])

#Trennung der Eintraege in der Spalte 'Wort' in zwei Teile, sicherstellen, dass immer zwei Teile zurückgegeben werden
df_most_common_words[['Typ', 'Wort']] = df_most_common_words['Wort'].apply(
    lambda x: pd.Series(x.split(':', 1) if ':' in x else [x, '']))

#Sortieren des DataFrame nach 'Anzahl im Text' in absteigender Reihenfolge
df_most_common_words = df_most_common_words.sort_values(by='Anzahl', ascending=False)

#Zuruecksetzen des Index
df_most_common_words = df_most_common_words.reset_index(drop=True)

#Entfernen von Sonderzeichen Spalten 'Typ' und 'Wert'
df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#Entfernen von Apostrophen in den Spalten 'Wort-Typ' und 'Wort-Wert'
df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace("'", "")
df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace("'", "")

#Filtern der Eintraege, wo sowohl 'Typ' als auch 'WWert' einen Eintrag haben
df_most_common_words = df_most_common_words[(df_most_common_words['Typ'] != '') & (df_most_common_words['Wort'] != '')]

#Neuordnen des DataFrame nach den Spalten 'Typ', 'Wort' und 'Anzahl'
df_most_common_words = df_most_common_words[['Typ', 'Wort', 'Anzahl']]

#Speichern des sortierten DataFrame in eine CSV-Datei
df_most_common_words.to_csv('4.1_NER_Entitaeten_vor_ChatGPT.csv', index=False)

#Oberen Teil des DataFrame anzeigen, um die Änderungen zu überprüfen
print(df_most_common_words.head(20))

C:\Users\MarkE\AppData\Local\Temp\ipykernel_4120\2397376143.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_vor_ChatGPT['NER_entitaeten'] = df_vor_ChatGPT['NER_entitaeten'].astype(str)


         Typ           Wort  Anzahl
0       MISC             AI     516
1   CARDINAL            one     134
2    ORDINAL          first     130
3   CARDINAL            two      80
4        GPE        Germany      72
5        ORG         Google      71
6       NORP         german      49
7       DATE          today      43
8    ORDINAL         second      40
9        GPE          China      35
10  CARDINAL          three      33
11   ORDINAL          third      28
12       GPE             US      28
13      NORP       european      26
14  CARDINAL           four      24
15      DATE           year      24
16      DATE           2022      24
17       GPE  United States      23
18      NORP        chinese      22
19       LOC         Europe      22


In [8]:
#Liste an Entitaeten im ersten Jahr nach ChatGPT
df_nach_ChatGPT = df[df['Zeitraum'] == '1. ChatGPT-Jahr']

#Sicherstellen, dass alle Werte in der Spalte als Strings behandelt werden
df_nach_ChatGPT['NER_entitaeten'] = df_nach_ChatGPT['NER_entitaeten'].astype(str)

#Woerter sind durch Kommas getrennt
words = df_nach_ChatGPT['NER_entitaeten'].str.split(',').explode().str.strip()

#Zähle die Woerter mit Counter
word_counts = Counter(words)

#Abrufen und Sortieren der häufigsten Woerter
most_common_words = word_counts.most_common()

#Umwandlung der Liste der häufigsten Woerter in einen DataFrame
df_most_common_words = pd.DataFrame(most_common_words, columns=['Wort', 'Anzahl'])

#Trennung der Eintraege in der Spalte 'Wort' in zwei Teile, sicherstellen, dass immer zwei Teile zurückgegeben werden
df_most_common_words[['Typ', 'Wort']] = df_most_common_words['Wort'].apply(
    lambda x: pd.Series(x.split(':', 1) if ':' in x else [x, '']))

#Sortieren des DataFrame nach 'Anzahl im Text' in absteigender Reihenfolge
df_most_common_words = df_most_common_words.sort_values(by='Anzahl', ascending=False)

#Zuruecksetzen des Index
df_most_common_words = df_most_common_words.reset_index(drop=True)

#Entfernen von Sonderzeichen Spalten 'Typ' und 'Wert'
df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#Entfernen von Apostrophen in den Spalten 'Wort-Typ' und 'Wort-Wert'
df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace("'", "")
df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace("'", "")

#Filtern der Eintraege, wo sowohl 'Typ' als auch 'WWert' einen Eintrag haben
df_most_common_words = df_most_common_words[(df_most_common_words['Typ'] != '') & (df_most_common_words['Wort'] != '')]

#Neuordnen des DataFrame nach den Spalten 'Typ', 'Wort' und 'Anzahl'
df_most_common_words = df_most_common_words[['Typ', 'Wort', 'Anzahl']]

#Speichern des sortierten DataFrame in eine CSV-Datei
df_most_common_words.to_csv('4.2_NER_Entitaeten_nach_ChatGPT_Jahr1.csv', index=False)

#Oberen Teil des DataFrame anzeigen, um die Änderungen zu überprüfen
print(df_most_common_words.head(20))

C:\Users\MarkE\AppData\Local\Temp\ipykernel_4120\765438366.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nach_ChatGPT['NER_entitaeten'] = df_nach_ChatGPT['NER_entitaeten'].astype(str)


         Typ        Wort  Anzahl
0       MISC          AI    6027
1    PRODUCT     Chatgpt    1225
2    ORDINAL       first     810
3   CARDINAL         one     722
4        ORG   Microsoft     716
5        GPE     Germany     638
6        ORG      Google     605
7        ORG      Openai     550
8     PERSON  Sam Altman     473
9       NORP      german     456
10      DATE       today     379
11  CARDINAL         two     379
12       GPE          US     366
13   PRODUCT         GPT     314
14  CARDINAL       three     262
15       ORG          EU     256
16    PERSON   Elon Musk     242
17       LOC      Europe     241
18   ORDINAL      second     228
19      NORP    european     192


In [9]:
#Liste an Entitaeten im zweiten Jahr nach ChatGPT
df_nach_ChatGPT = df[df['Zeitraum'] == '2. ChatGPT-Jahr']

#Sicherstellen, dass alle Werte in der Spalte als Strings behandelt werden
df_nach_ChatGPT['NER_entitaeten'] = df_nach_ChatGPT['NER_entitaeten'].astype(str)

#Woerter sind durch Kommas getrennt
words = df_nach_ChatGPT['NER_entitaeten'].str.split(',').explode().str.strip()

#Zähle die Woerter mit Counter
word_counts = Counter(words)

#Abrufen und Sortieren der häufigsten Woerter
most_common_words = word_counts.most_common()

#Umwandlung der Liste der häufigsten Woerter in einen DataFrame
df_most_common_words = pd.DataFrame(most_common_words, columns=['Wort', 'Anzahl'])

#Trennung der Eintraege in der Spalte 'Wort' in zwei Teile, sicherstellen, dass immer zwei Teile zurückgegeben werden
df_most_common_words[['Typ', 'Wort']] = df_most_common_words['Wort'].apply(
    lambda x: pd.Series(x.split(':', 1) if ':' in x else [x, '']))

#Sortieren des DataFrame nach 'Anzahl im Text' in absteigender Reihenfolge
df_most_common_words = df_most_common_words.sort_values(by='Anzahl', ascending=False)

#Zuruecksetzen des Index
df_most_common_words = df_most_common_words.reset_index(drop=True)

#Entfernen von Sonderzeichen Spalten 'Typ' und 'Wert'
df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

#Entfernen von Apostrophen in den Spalten 'Wort-Typ' und 'Wort-Wert'
df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace("'", "")
df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace("'", "")

#Filtern der Eintraege, wo sowohl 'Typ' als auch 'WWert' einen Eintrag haben
df_most_common_words = df_most_common_words[(df_most_common_words['Typ'] != '') & (df_most_common_words['Wort'] != '')]

#Neuordnen des DataFrame nach den Spalten 'Typ', 'Wort' und 'Anzahl'
df_most_common_words = df_most_common_words[['Typ', 'Wort', 'Anzahl']]

#Speichern des sortierten DataFrame in eine CSV-Datei
df_most_common_words.to_csv('4.3_NER_Entitaeten_nach_ChatGPT_Jahr2.csv', index=False)

#Oberen Teil des DataFrame anzeigen, um die Änderungen zu überprüfen
print(df_most_common_words.head(20))

C:\Users\MarkE\AppData\Local\Temp\ipykernel_4120\763238643.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_nach_ChatGPT['NER_entitaeten'] = df_nach_ChatGPT['NER_entitaeten'].astype(str)


         Typ       Wort  Anzahl
0       MISC         AI    4224
1    ORDINAL      first     541
2        GPE    Germany     504
3   CARDINAL        one     431
4        ORG  Microsoft     376
5    PRODUCT    Chatgpt     348
6       NORP     german     322
7        ORG     Google     317
8   CARDINAL        two     266
9        ORG         EU     253
10       ORG     Nvidia     209
11      DATE      today     204
12       GPE         US     193
13       ORG     Openai     184
14       LOC     Europe     170
15   ORDINAL     second     164
16       GPE      China     155
17      NORP   american     147
18       ORG      Apple     147
19      NORP   european     138


In [10]:
#Speichern des geänderten DataFrame in einer neuen CSV-Datei
df.to_csv('5_Daten_nach_NER.csv', index=False)

In [11]:
#Liste an Entitaeten nach Zeitraum und Nachrichtenportale für weitere Analyse der Entwicklung

#Sicherstellen, dass alle Werte in der Spalte als Strings behandelt werden
df['NER_entitaeten'] = df['NER_entitaeten'].astype(str)

#Erstellen eines leeren DataFrames für die detaillierten Entitäten
df_entitaeten_details = pd.DataFrame()

#Gruppieren der Daten nach 'Jahr.Monat' und 'Quelle'
grouped = df.groupby(['Jahr.Monat', 'Quelle'])

#Verarbeitung jeder Gruppe
for (year_month, source), group in grouped:
    #Woerter sind durch Kommas getrennt
    words = group['NER_entitaeten'].str.split(',').explode().str.strip()

    #Zaehle die Woerter mit Counter
    word_counts = Counter(words)

    #Abrufen und Sortieren der haeufigsten Woerter
    most_common_words = word_counts.most_common()

    #Umwandlung der Liste der haeufigsten Woerter in einen DataFrame
    df_most_common_words = pd.DataFrame(most_common_words, columns=['Wort', 'Anzahl'])

    #Trennung der Eintraege in der Spalte 'Wort' in Typ und Wert
    df_most_common_words[['Typ', 'Wort']] = df_most_common_words['Wort'].apply(
        lambda x: pd.Series(x.split(':', 1) if ':' in x else [x, '']))

    #Bereinigen der Spalten 'Typ' und 'Wort'
    df_most_common_words['Typ'] = df_most_common_words['Typ'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)
    df_most_common_words['Wort'] = df_most_common_words['Wort'].str.replace(r'[^a-zA-Z0-9\s]', '', regex=True)

    #Filtern der Eintraege
    df_most_common_words = df_most_common_words[(df_most_common_words['Typ'] != '') & (df_most_common_words['Wort'] != '')]

    #Hinzufuegen der Jahr.Monat und Quelle Informationen
    df_most_common_words['Jahr.Monat'] = year_month
    df_most_common_words['Quelle'] = source

    #Hinzufügen der Ergebnisse zum Gesamt-DataFrame
    df_entitaeten_details = pd.concat([df_entitaeten_details, df_most_common_words], ignore_index=True)

#Sortieren und Speichern des sortierten DataFrames
df_entitaeten_details = df_entitaeten_details[['Jahr.Monat', 'Quelle','Typ', 'Wort', 'Anzahl']]

#Konvertieren der Spalte 'Jahr.Monat' in Perioden
df_entitaeten_details['Jahr.Monat'] = pd.PeriodIndex(df_entitaeten_details['Jahr.Monat'], freq='M')

#Kategorisierung nach Zeitraum
def categorize_period(date):
    if date <= pd.Period('2022-11', 'M'):
        return '0. vor ChatGPT'
    elif pd.Period('2022-12', 'M') <= date <= pd.Period('2023-11', 'M'):
        return '1. ChatGPT-Jahr'
    elif pd.Period('2023-12', 'M') <= date <= pd.Period('2024-11', 'M'):
        return '2. ChatGPT-Jahr'
    return 'Andere'

df_entitaeten_details['Zeitraum'] = df_entitaeten_details['Jahr.Monat'].apply(categorize_period)

#Anzeige DataFrame, um die Aenderungen zu überprüfen
print(df_entitaeten_details.head(20))

   Jahr.Monat Quelle       Typ         Wort  Anzahl        Zeitraum
0     2021-12    FAZ      MISC           AI      30  0. vor ChatGPT
1     2021-12    FAZ       GPE      Germany      10  0. vor ChatGPT
2     2021-12    FAZ  CARDINAL          one       8  0. vor ChatGPT
3     2021-12    FAZ   ORDINAL        first       6  0. vor ChatGPT
4     2021-12    FAZ       GPE        Kaust       5  0. vor ChatGPT
5     2021-12    FAZ      NORP       german       4  0. vor ChatGPT
6     2021-12    FAZ  CARDINAL          two       3  0. vor ChatGPT
7     2021-12    FAZ       LOC      Red Sea       2  0. vor ChatGPT
8     2021-12    FAZ       GPE  Switzerland       2  0. vor ChatGPT
9     2021-12    FAZ     EVENT       Corona       2  0. vor ChatGPT
10    2021-12    FAZ      DATE         2020       2  0. vor ChatGPT
11    2021-12    FAZ       ORG       Racoon       2  0. vor ChatGPT
12    2021-12    FAZ       ORG       Bucher       2  0. vor ChatGPT
13    2021-12    FAZ       GPE       Corona     

In [12]:
#Entitaetenanalyse -  Entitaeten vor und nach ChatGPT

#Aggregiere den DataFrame nach "Zeitraum", "Wort" und berechne die Summe der Anzahl im Text
df_entitaeten_nach_Zeit = df_entitaeten_details.groupby(['Zeitraum', 'Typ','Wort']).agg(
    Summe_Anzahl_im_Text=('Anzahl', 'sum'),
).reset_index()

#Sortiere den DataFrame nach "Zeitraum" aufsteigend
df_entitaeten_nach_Zeit = df_entitaeten_nach_Zeit.sort_values(by='Zeitraum', ascending=True)

#Umformatieren des DataFrames, um die Summen zu erhalten
df_pivot = df_entitaeten_nach_Zeit.pivot_table(index=['Typ','Wort'], columns='Zeitraum', values=['Summe_Anzahl_im_Text'])

#Ersetze NaN-Werte durch 0
df_pivot.fillna(0, inplace=True)

#Flatten des Multi-Index, um die CSV-Datei zu vereinfachen
df_pivot.columns = [f'{col[0]}_{col[1]}' for col in df_pivot.columns]

#Sortiere den DataFrame nach der Spalte "Summe_Anzahl_im_Text "1. ChatGPT-Jahr" absteigend
df_pivot = df_pivot.sort_values(by='Summe_Anzahl_im_Text_2. ChatGPT-Jahr', ascending=False)

#Anzeigen der bearbeiteten Tabelle
print(df_pivot)

df_pivot.to_csv('6_Entitaeten_vor_nach_ChatGPT.csv')

                                             Summe_Anzahl_im_Text_0. vor ChatGPT  \
Typ       Wort                                                                     
MISC      AI                                                               543.0   
ORDINAL   first                                                            138.0   
GPE       Germany                                                           74.0   
CARDINAL  one                                                              141.0   
ORG       Microsoft                                                         16.0   
...                                                                          ...   
          Washington Post 30000                                              0.0   
          Washington Post Guardian                                           1.0   
          Washington Post New York Times                                     0.0   
          Washington Post Patentsgooglecom                                  

In [13]:
#Entitaetenanalyse -  Entitaeten nach Monaten

#Aggregiere den DataFrame nach "Zeitraum", "Wort" und berechne die Summe der Anzahl im Text
df_entitaeten_nach_Zeit = df_entitaeten_details.groupby(['Jahr.Monat', 'Typ','Wort']).agg(
    Summe_Anzahl_im_Text=('Anzahl', 'sum'),
).reset_index()

#Sortiere den DataFrame nach "Zeitraum" aufsteigend
df_entitaeten_nach_Zeit = df_entitaeten_nach_Zeit.sort_values(by='Jahr.Monat', ascending=True)

#Umformatieren des DataFrames, um die Summen zu erhalten
df_pivot = df_entitaeten_nach_Zeit.pivot_table(index=['Typ','Wort'], columns='Jahr.Monat', values=['Summe_Anzahl_im_Text'])

#Ersetze NaN-Werte durch 0
df_pivot.fillna(0, inplace=True)

#Flatten des Multi-Index, um die CSV-Datei zu vereinfachen
df_pivot.columns = [f'{col[0]}_{col[1]}' for col in df_pivot.columns]

#Sortiere den DataFrame nach der Spalte "Summe_Anzahl_im_Text_2023-11" absteigend
df_pivot = df_pivot.sort_values(by='Summe_Anzahl_im_Text_2023-11', ascending=False)

#Anzeigen der bearbeiteten Tabelle
print(df_pivot.head(10))

df_pivot.to_csv('7_Entitaeten_Monaten.csv')

                     Summe_Anzahl_im_Text_2021-12  \
Typ      Wort                                       
MISC     AI                                  67.0   
PERSON   Sam Altman                           0.0   
ORG      Openai                               2.0   
         Microsoft                            2.0   
PRODUCT  Chatgpt                              0.0   
GPE      Germany                             17.0   
CARDINAL one                                 14.0   
ORDINAL  first                               16.0   
PERSON   Elon Musk                            0.0   
CARDINAL two                                  9.0   

                     Summe_Anzahl_im_Text_2022-01  \
Typ      Wort                                       
MISC     AI                                  30.0   
PERSON   Sam Altman                           0.0   
ORG      Openai                               1.0   
         Microsoft                            1.0   
PRODUCT  Chatgpt                             